In [ ]:
#pip install gensim

In [ ]:
def preprocess(text):
   text = text.lower()
   doc = word_tokenize(text)
   doc = [word for word in doc if word not in stop_words]
   doc = [word for word in doc if word.isalpha()]
   return doc

texts, y = ng20.data, ng20.target
               
corpus = [preprocess(text) for text in texts]

In [ ]:
import numpy as np
from gensim import corpora
from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity

sims = {'ng20': {}, 'snippets': {}}
dictionary = corpora.Dictionary(corpus)
corpus_gensim = [dictionary.doc2bow(doc) for doc in corpus]
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]
lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=200)
lsi_index = MatrixSimilarity(lsi[corpus_tfidf])
sims['ng20']['LSI'] = np.array([lsi_index[lsi[corpus_tfidf[i]]]
                                for i in range(len(corpus))])


In [1]:
#Import job data
import pandas as pd
from pandas import DataFrame
import json

data = []

with open("/Users/ryancramer/Desktop/handshake_feed1.txt") as f:
    for line in f:
        data.append(json.loads(line))
        
df1 = pd.DataFrame(data)

data = []

with open("/Users/ryancramer/Desktop/courses.txt") as f:
    for line in f:
        data.append(json.loads(line))
        
df2 = pd.DataFrame(data)



#train_texts = df4['description']

In [4]:
jobs = df1['description']
courses = df2['description']

In [2]:
import os.path
from gensim import corpora
#from gensim.models import TfidfModel
#from gensim.models import LsiModel
#from gensim.similarities import MatrixSimilarity

from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

import numpy as np
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity

In [1]:
def preprocess_data(doc_set):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        alpha_tokens = [i for i in stopped_tokens if i.isalpha()]
        # alpha tokens only
        stemmed_tokens = [p_stemmer.stem(i) for i in alpha_tokens]
        # add tokens to list
        texts.append(stemmed_tokens)
    return texts

In [2]:
%%time
jobs = preprocess_data(jobs)



NameError: name 'jobs' is not defined

In [6]:
%%time
courses = preprocess_data(courses)

CPU times: user 38.4 s, sys: 585 ms, total: 39 s
Wall time: 42.9 s


In [7]:
print(len(jobs))
print(len(courses))

876
8781


In [8]:
def prepare_corpus(doc_clean):
    """
    Input  : clean document
    Purpose: create term dictionary of our courpus and Converting list of documents (corpus) into Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix

In [ ]:
len(jobs)

In [9]:
for course in courses:
    jobs.append(course)

In [ ]:
df3 = pd.read_csv("/Users/ryancramer/Desktop/Jupyter Notebook Files/first_write.csv")
    
Simple_Skills = df3["0"]


In [ ]:
skill = preprocess_data(Simple_Skills)

In [ ]:
skillss = list()
for word in skill:
    for skills in word:
        if skills not in skillss:
            skillss.append(skills)

In [ ]:
dictionary23, doc_term_matrix23 = prepare_corpus(skill)

In [10]:
#jobs simularity
import gensim
sims = {'jobs': {}, 'courses': {}}

dictionary, doc_term_matrix = prepare_corpus(jobs)
lsimodel = LsiModel(doc_term_matrix, num_topics=8, id2word = dictionary)  # train model
#print(lsimodel.print_topics(num_topics=4, num_words=10))

tfidf = TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]


#
#sims = gensim.similarities.Similarity(jobs,corpus_tfidf, num_features=len(dictionary))


#

lsi_index = MatrixSimilarity(lsimodel[corpus_tfidf])


sims['jobs']['LSI'] = np.array([lsi_index[lsimodel[corpus_tfidf[i]]] for i in range(len(jobs))])


In [ ]:
#courses simularity
dictionary_courses, doc_term_matrix_courses = prepare_corpus(courses)
lsamodel_courses = LsiModel(doc_term_matrix_courses, num_topics=8, id2word = dictionary_courses)  # train model
#print(lsamodel.print_topics(num_topics=4, num_words=10))


tfidf2 = TfidfModel(doc_term_matrix_courses)
corpus_tfidf2 = tfidf2[doc_term_matrix_courses]


lsi_index_courses = MatrixSimilarity(lsamodel_courses[corpus_tfidf2])

sims['courses']['LSI'] = np.array([lsi_index_courses[lsamodel_courses[corpus_tfidf2[i]]] for i in range(len(courses))])

In [ ]:
sims

In [ ]:
courses[0]

In [ ]:
query_doc = [w.lower() for w in (courses[0])]
#print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
#print(query_doc_bow)
query_doc_tf_idf = tfidf[query_doc_bow]
print(query_doc_tf_idf)

In [ ]:
print(sims[query_doc_tf_idf])

In [ ]:

def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    #num_topics = 5
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=10, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
#compute_coherence_values(dictionary, doc_term_matrix, jobs, 10)

In [ ]:
def plot_graph(doc_clean,start, stop, step):
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    model_list, coherence_values = compute_coherence_values(dictionary, doc_term_matrix,jobs,
                                                            stop, start, step)
    # Show graph
    x = range(start, stop, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Number of Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [ ]:

#start,stop,step=2,10,1
#plot_graph(jobs,start,stop,step)

In [ ]:
len(courses)

In [ ]:
len(sims['courses']['LSI'][0])

In [11]:
#print(jobs[0])
count = 0
hold = 6491-876
for word in jobs[0]:
    if word in courses[(hold)]:
        
        count = count +1

#print(count/len(jobs[15]))
#print(courses[244])
print(count)

23


In [12]:
for word in courses[hold]:
    print(word)
    if word in skill:
        print("1")

aspir


NameError: name 'skill' is not defined

In [ ]:
print(courses[hold])
print(jobs[0])

### 

In [ ]:
print(jobs[15])
print(courses[0])

In [16]:
#######
r = np.argsort(sims['jobs']['LSI'][0])[::-1]
#print(r)
count = 0
for word in r:
    if word > 876:
        print(word) 
        count = count + 1
        if count == 20:
            break

#print(r)

1860
7226
3926
8606
4158
2646
4116
9115
6106
1135
3536
8220
8346
1280
8684


In [ ]:
r = np.argsort(sims['jobs']['LSI'][0])[::-1]
#print(r)
count = 0
for word in r:
    if word > 876:
        print(word)
        count = count + 1

print(r)

In [ ]:
def most_similar(i, X_sims, topn=None):
    """return the indices of the topn most similar documents with document i
    given the similarity matrix X_sims"""
               
    r = np.argsort(X_sims[i])[::-1]
    if r is None:
        return r
    else:
        return r[:topn]
               
#LSI
#print(sims['jobs']['LSI'][4])
jobs_similar = most_similar(4, sims['jobs']['LSI'], 10)
courses_similar = most_similar(4, sims['courses']['LSI'], 10)

In [ ]:
print(jobs_similar[4])

In [ ]:
print(courses[467])
print(courses[8687])

In [ ]:
r = np.argsort(sims['jobs']['LSI'][4])[::-1]
print(r)
print(r[:10])


In [ ]:
jobs_similar


In [ ]:
len(jobs)

In [ ]:
for job in jobs_similar:
    print(job)

#for job in courses_similar:
 #   print(job)

In [ ]:
print(jobs[4])
print(jobs[600])

In [ ]:
for job in jobs_similar:
    print(job)

for job in courses_similar:
    print(job)

In [ ]:
print(courses[0])
print(courses[3310])

In [ ]:

for i, s in enumerate(sims['courses']['LSI']):
    print(s, sims['courses']['LSI'][i])

In [ ]:
lsi_index.print_topics(num_topics=4, num_words=10)

In [ ]:
def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel